# Table of Contents

1. Calculate Molar Mass
2. Balance Chemical Equation
3. Chemical Equilibria

ref.: https://pypi.org/project/chempy/#toc-entry-6

***

In [3]:
!pip install chempy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.7/184.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.8 MB/s eta 0:00:00
  Created wheel for chempy: filename=chempy-0.9.0-py2.py3-none-any.whl size=210812 sha256=d27658d3b16301a50063f497ec9f3410a3ab4965be294562840b94fd7f06c3b9
  Stored in directory: /root/.cache/pip/wheels/67/6a/c8/0b931e3c1c6c4e620af52ab72afb8377af84307fc715d3d72b
  Created wheel for pyneqsys: filename=pyneqsys-0.5.7-py2.py3-none-any.whl size=27262 sha256=c58401cbdb59418bee8dcb74b9621eb93983be4ed0abb6937eabd7cae848475b
  Stored in dire


1. Calculate Molar Mass


In [17]:
from chempy import Substance
import quantities as q

def calculate_substance_properties(formula):
  """Calculates and prints properties of a chemical substance.

  Args:
    formula: The chemical formula of the substance (e.g., 'Fe(CN)6-3').

  Returns:
    None. substance's unicode name and molar mass in g/mol.
  """
  substance = Substance.from_formula(formula)
  mass_with_units = substance.mass * q.gram / q.mol  # Store mass with units

  return(substance.unicode_name, mass_with_units)


# Example usage:
chemical_formula = "H2SO4"

unicode_formula, molar_mass = calculate_substance_properties(chemical_formula)
print("Unicode formula: %s" %(unicode_formula))
print("Molar mass: %s" %(molar_mass))

Unicode formula: H₂SO₄
Molar mass: 98.072 g/mol


2. Balance Chemical Equation


In [35]:
from chempy import balance_stoichiometry
from chempy import Substance
import quantities as q

def balance_chemical_equation(reaction_string):
    """
    Balances a chemical equation using chempy.

    Parameters:
    reaction_string (str): A string representing the chemical equation, e.g., "H2 + O2 -> H2O".

    Returns:
    str: A string representing the balanced chemical equation or an error message if balancing fails.
    """
    try:
        # Parse reactants and products and split by "->"
        reactants_str, products_str = reaction_string.split("->")
        reactants_str = reactants_str.strip()
        products_str = products_str.strip()

        # Parse reactants by "+" and coefficients by spaces
        reactants = {}
        for part in reactants_str.split("+"):
            part = part.strip()
            if part:
                if " " in part:
                    coeff, molecule = part.split()
                    reactants[molecule] = int(coeff)
                else:
                    reactants[part] = 1

        # Parse products by "+" and coefficients by spaces
        products = {}
        for part in products_str.split("+"):
            part = part.strip()
            if part:
                if " " in part:
                    coeff, molecule = part.split()
                    products[molecule] = int(coeff)
                else:
                    products[part] = 1

        reactants, products = balance_stoichiometry(reactants, products)

        balanced_reaction_string = " + ".join([f"{coeff}{' '}{molecule}" for molecule, coeff in reactants.items()]) \
                                  + " -> " \
                                  + " + ".join([f"{coeff}{' '}{molecule}" for molecule, coeff in products.items()])

        return balanced_reaction_string

    except Exception as e:
        return f"Error balancing equation: {e}"

# Example usage:
reaction_string = "3 H2 + O2 -> H2O"
balanced_reaction_string = balance_chemical_equation(reaction_string)

print(balanced_reaction_string)

2 H2 + 1 O2 -> 2 H2O


3. Chemical Equilibria

In [34]:
from collections import defaultdict
from chempy.equilibria import EqSystem
from math import log10

def calculate_equilibrium_and_ph(initial_concentrations, equilibrium_expression):
    """
    Calculate equilibrium concentrations and pH from given initial concentrations and equilibrium expression.

    Parameters:
    initial_concentrations (dict): A dictionary containing the initial concentrations of all species involved in the equilibrium.
    equilibrium_expression (str): The equilibrium expression in the format 'A = B + C; K'.

    Returns:
    dict: A dictionary containing the equilibrium concentrations of all species.
    float: The pH of the solution.
    float: The H+ concentration at equilibrium.
    """
    # Define the equilibrium system
    eqsys = EqSystem.from_string(equilibrium_expression)

    # Solve for equilibrium concentrations
    arr, info, sane = eqsys.root(initial_concentrations)
    conc = dict(zip(eqsys.substances, arr))

    # Calculate pH and H+ concentration
    pH = -log10(conc['H+'])
    h_concentration = conc['H+']

    return conc, pH, h_concentration

# Example usage:
initial_conditions = defaultdict(float, {'HC2H3O2': 0.2, 'H+': 1e-7, 'C2H3O2-': 0.1})
equilibrium_eq = "HC2H3O2 = H+ + C2H3O2-; 1.8*10**-5"
equilibrium_concentrations, pH_value, h_plus_concentration = calculate_equilibrium_and_ph(initial_conditions, equilibrium_eq)

print("Equilibrium Concentrations:", equilibrium_concentrations)
print("pH: %.2f" % pH_value)
print("H+ concentration: %.2e" % h_plus_concentration)

Equilibrium Concentrations: {'C2H3O2-': 0.10003588063140861, 'H+': 3.598063140861223e-05, 'HC2H3O2': 0.1999641193685914}
pH: 4.44
H+ concentration: 3.60e-05


***